# MLflow 

The sktime custom model flavor enables logging of sktime models in MLflow format via the `sktime.utils.mlflow_sktime.save_model()` and `sktime.utils.mlflow_sktime.log_model()` methods. These methods also add the `pyfunc` flavor to the MLflow Models that they produce, allowing the model to be interpreted as generic Python functions for inference via `sktime.utils.mlflow_sktime.pyfunc.load_model()`. This loaded PyFunc model can only be scored with a DataFrame input. You can also use the `sktime.utils.mlflow_sktime.load_model()` method to load MLflow Models with the sktime model flavor in native sktime formats.

The `pyfunc` flavor of the model supports sktime predict methods `predict`,  `predict_interval`, `predict_quantiles` and `predict_var` for scitypes `"Series"` and `"Panel"`.

The interface for utilizing a sktime model loaded as a `pyfunc` type for generating forecasts uses a *single-row* Pandas DataFrame configuration argument. The following columns in this configuration Pandas DataFrame are supported:


* ``predict_method`` (optional) - a string (default: ``predict``) specifies the sktime predict method. 
    For example, if the objective is to compute prediction interval forecasts, set the column ``predict_method`` to ``predict_interval``.
* ``X`` (optional) - exogenous regressor values as 2D or 3D numpy array of values for future time period events (default: ``None``). 
* ``fh`` (optional) - the forecasting horizon encoding the time stamps to be forecasted, optional (default: ``None``). 
* ``coverage`` (optional) - nominal coverage(s) of predictive interval(s) (default: ``0.90``), can only be used with ``predict_method=predict_interval``.
* ``alpha`` (optional) - A probability or list of, at which quantile forecasts are computed (default: ``None``), can only be used with ``predict_method=predict_quantiles``.
* ``cov`` (optional) - if True, computes covariance matrix forecast. if False, computes marginal variance forecasts. (default: ``False``), can only be used with ``predict_method=predict_var``.

An example configuration for the ``pyfunc`` prediction interval forecast of a sktime model is shown below, with a forecast horizon of one to three periods and two prediction intervals:

| predict_method      | fh | coverage    |
| :---        |    :----   |          :--- |
| predict_interval      | [1, 2, 3]      | [0.1, 0.9]   |

Signature logging for sktime from a non-pyfunc artifact will not function correctly for `predict_interval` or `predict_quantiles`. The output of the native sktime model flavor for these methods is not a recognized signature type due to the MultiIndex column structure of the returned DataFrame. MLflow's ``infer_schema`` will function correctly if using the ``pyfunc`` flavor of the model, though.

## 1. Setup
### 1.1 Config

In [23]:
model_path = "model"

### 1.1 Imports

In [24]:
import mlflow
import pandas as pd

from sktime.datasets import load_longley
from sktime.datatypes import convert
from sktime.forecasting.model_selection import temporal_train_test_split
from sktime.forecasting.naive import NaiveForecaster
from sktime.utils import mlflow_sktime

### 1.2 Load sample data

In [25]:
y, X = load_longley()
y_train, y_test, X_train, X_test = temporal_train_test_split(y, X)

# For utilizing sktime model in pyfunc flavor the exogenous regressor must be passed as a numpy array to configuration DataFrame  # noqa: E501
X_test_array = convert(X_test, "pd.DataFrame", "np.ndarray")

## 2. Example usage of `mlflow_sktime.save_model()`

In [26]:
with mlflow.start_run():

    forecaster = NaiveForecaster()
    forecaster.fit(y_train, X_train)

    mlflow_sktime.save_model(forecaster, model_path)

# 3. Loading model in different flavors

### 3.1 Native sktime flavor

In [27]:
loaded_model = mlflow_sktime.load_model(model_path)

### 3.2 Pyfunc flavor

In [ ]:
loaded_pyfunc = mlflow_sktime.pyfunc.load_model(model_path)

## 4. Example usage of sktime `predict()`

### 4.1 Native sktime flavor

In [ ]:
predictions = loaded_model.predict(fh=[1, 2, 3], X=X_test)
predictions

### 4.2 Pyfunc flavor

In [ ]:
prediction_conf = pd.DataFrame(
    [
        {
            "predict_method": "predict",
            "fh": [1, 2, 3],
            "coverage": [0.1, 0.9],
            "X": X_test_array,
        }
    ]
)
predictions = loaded_pyfunc.predict(prediction_conf)
predictions

## 5. Example usage of sktime `predict_interval()`

### 5.1 Native sktime flavor

In [ ]:
predictions = loaded_model.predict_interval(fh=[1, 2, 3], coverage=[0.1, 0.9], X=X_test)
predictions

### 5.2 Pyfunc flavor

In [ ]:
prediction_conf = pd.DataFrame(
    [
        {
            "predict_method": "predict_interval",
            "fh": [1, 2, 3],
            "coverage": [0.1, 0.9],
            "X": X_test_array,
        }
    ]
)
predictions = loaded_pyfunc.predict(prediction_conf)
predictions

## 6. Example usage of sktime `predict_quantiles()`

### 6.1 Native sktime flavor

In [ ]:
predictions = loaded_model.predict_quantiles(
    fh=[1, 2, 3], alpha=[0.1, 0.5, 0.9], X=X_test
)
predictions

### 6.2 Pyfunc flavor

In [ ]:
prediction_conf = pd.DataFrame(
    [
        {
            "predict_method": "predict_quantiles",
            "fh": [1, 2, 3],
            "alpha": [0.1, 0.5, 0.9],
            "X": X_test_array,
        }
    ]
)
predictions = loaded_pyfunc.predict(prediction_conf)
predictions

## 7. Example usage of sktime `predict_var()`

### 7.1 Native sktime flavor

In [28]:
predictions = loaded_model.predict_var(fh=[1, 2, 3], X=X_test)
predictions

,0
1959,1.957628e+06
1960,3.915256e+06
1961,5.872885e+06


### 7.2 Pyfunc flavor

In [29]:
prediction_conf = pd.DataFrame(
    [{"predict_method": "predict_var", "fh": [1, 2, 3], "X": X_test_array}]
)
predictions = loaded_pyfunc.predict(prediction_conf)
predictions

,0
1959,1.957628e+06
1960,3.915256e+06
1961,5.872885e+06


## 8. Model deployment example

### 8.1 Create experiment

In [93]:
artifact_path = "model"

mlflow.set_experiment("Test Sktime")

with mlflow.start_run() as run:

    forecaster = NaiveForecaster()
    forecaster.fit(y_train, X_train)

    mlflow_sktime.log_model(sktime_model=forecaster, artifact_path=artifact_path)

run_id = run.info.run_id
print(f"MLflow run id: {run_id}")

MLflow run id: 99b7ea689b0843889e3b181030fc44cf


### 8.2 Deploy pyfunc model to local REST API endpoint
- Open a terminal window and cd into `examples`directory
- In the terminal run: `mlflow models serve -m runs:/<RUN_ID>/model --env-manager local --host <HOST>`
    - where you substitute `<RUN_ID>` by the `run_id` and `<HOST>` by the network address to listen on (e.g. `127.0.0.2`) 
- More details here: https://www.mlflow.org/docs/latest/cli.html#mlflow-models-serve

### 8.3 Request predictions from local REST API endpoint

- For mor details see: https://www.mlflow.org/docs/latest/models.html#built-in-deployment-tools

#### 8.3.1 JSON input using `dataframe_split` field with pandas DataFrame in the `split` orientation

In [88]:
# host = "127.0.0.2"
# url = f"http://{host}:5000/invocations"

# prediction_conf = pd.DataFrame([{"predict_method": "predict", "fh": 1}])
# json_data = {"dataframe_split": prediction_conf.to_dict(orient="split")}
# print(json_data)

# import requests
# response = requests.post(url, json=json_data)
# response.json()

{'dataframe_split': {'index': [0], 'columns': ['predict_method', 'fh'], 'data': [['predict', 1]]}}


{'predictions': [{'0': 66513.0}]}

#### 8.3.1 JSON input using `dataframe_records` field with pandas DataFrame in the `records` orientation

In [91]:
# json_data = {
#     "dataframe_records": prediction_conf.to_dict(orient="records")
# }
# print(json_data)

# response = requests.post(url, json=json_data)
# response.json()

{'dataframe_records': [{'predict_method': 'predict', 'fh': 1}]}


{'predictions': [{'0': 66513.0}]}

#### 8.3.2 CSV input using valid `pd.DataFrame` csv representation

In [92]:
# headers = {
#     "Content-Type": "text/csv",
# }
# data = prediction_conf.to_csv()
# print(data)

# response = requests.post(url, headers=headers, data=data)
# response.json()

,predict_method,fh
0,predict,1



{'predictions': [{'0': 66513.0}]}